In [ ]:
import pandas as pd 
import numpy as np
import glob
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop , SGD
from keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint , ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip,RandomRotation
from tensorflow.keras.layers import Dense , Input , Conv2D  ,MaxPooling2D ,Flatten , Lambda ,Average, UpSampling2D ,Conv2DTranspose ,Reshape
from tensorflow.keras.applications.vgg19 import VGG19
from keras.utils import to_categorical
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
train.head()

In [ ]:
train.target.value_counts()

In [ ]:
paths = train[train.target == 0].image_name.values
paths.shape

In [ ]:
# data_augmentation

data_augmentation = tf.keras.Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
])

In [ ]:
# AE encoder 
'''
input shape = (256,256,3)
'''

input_shape = (256,256,3)

input_encoder = Input(input_shape,name = 'encoder_input')
encoder = Conv2D(16,(3,3),activation = 'relu',name = 'encoder_layer1')(input_encoder)
encoder = Conv2D(16,(3,3),activation = 'relu',name = 'encoder_layer1_1')(encoder)
encoder = Conv2D(32,(3,3),activation = 'relu',name = 'encoder_layer1_2')(encoder)
encoder = MaxPooling2D(2,2 , name = 'encoder_layer2')(encoder)
encoder = Conv2D(32,(3,3),activation = 'relu',name = 'encoder_layer3')(encoder)
encoder = Conv2D(32,(3,3),activation = 'relu',name = 'encoder_layer3_1')(encoder)
encoder = Conv2D(32,(3,3),activation = 'relu',name = 'encoder_layer3_2')(encoder)
encoder = MaxPooling2D(3,3 , name = 'encoder_layer4')(encoder)
encoder = Conv2D(64,(3,3),activation = 'relu',name = 'encoder_layer5')(encoder)
encoder = Conv2D(64,(3,3),activation = 'relu',name = 'encoder_layer5_1')(encoder)
encoder = Conv2D(64,(4,4),activation = 'relu',name = 'encoder_layer5_2')(encoder)
encoder = Conv2D(64,(4,4),activation = 'relu',name = 'encoder_layer5_3')(encoder)
encoder = MaxPooling2D(3,3 , name = 'encoder_layer6')(encoder)
encoder = Flatten(name = 'encoder_layer7')(encoder)
encoder = Dense(256 , activation = 'relu', name = 'encoder_layer8')(encoder)

Encoder = Model(inputs= [input_encoder], outputs=[encoder],name = 'Encoder')
print(Encoder.summary())
plot_model(Encoder, show_shapes=True)

In [ ]:
#AE decoder

'''
input shape = (256)
'''

input_shape = 256

input_decoder = Input(input_shape,name = 'decoder_input')
decoder = Dense((5184),name = 'decoder_layer1_1')(input_decoder)
decoder = Reshape((9, 9, 64),name = 'decoder_layer1_2')(decoder)
decoder = UpSampling2D((3,3),name = 'decoder_layer1_3')(decoder)
decoder = Conv2DTranspose(64, (3,3), activation='relu' , name = 'decoder_layer1_4')(decoder)
decoder = Conv2DTranspose(64, (4,4), activation='relu' , name = 'decoder_layer2')(decoder)
decoder = Conv2DTranspose(64, (4,4), activation='relu' , name = 'decoder_layer2_1')(decoder)
decoder = Conv2DTranspose(64, (3,3), activation='relu' , name = 'decoder_layer2_2')(decoder)
decoder = Conv2DTranspose(64, (3,3), activation='relu' , name = 'decoder_layer2_3')(decoder)
decoder = UpSampling2D((3,3),name = 'decoder_layer3')(decoder)
decoder = Conv2DTranspose(32, (3,3), activation='relu' , name = 'decoder_layer3_1')(decoder)
decoder = Conv2DTranspose(32, (3,3), activation='relu' , name = 'decoder_layer4')(decoder)
decoder = Conv2DTranspose(32, (3,3), activation='relu' , name = 'decoder_layer4_1')(decoder)
decoder = Conv2DTranspose(32, (3,3), activation='relu' , name = 'decoder_layer4_2')(decoder)
decoder = UpSampling2D((2,2),name = 'decoder_layer5')(decoder)
decoder = Conv2DTranspose(32, (3,3), activation='relu' , name = 'decoder_layer6')(decoder)
decoder = Conv2DTranspose(16, (3,3), activation='relu' , name = 'decoder_layer6_1')(decoder)
decoder = Conv2DTranspose(16, (3,3), activation='relu' , name = 'decoder_layer6_2')(decoder)
output_decoder = Conv2D(3,(1,1), name = 'decoder_layer9' , activation= 'relu')(decoder)

Decoder = Model(inputs= [input_decoder], outputs=[output_decoder],name = 'Decoder')
print(Decoder.summary())
plot_model(Decoder, show_shapes=True)

In [ ]:
AE_input = Input((256,256,3),name = 'AE_input')
AE_latent = Encoder(AE_input)
AE_output = Decoder(AE_latent)
AE = Model(inputs= [AE_input], outputs=[AE_output],name = 'AE')

print(AE.summary())
plot_model(AE, show_shapes=True)

In [ ]:
'''
define perceptual_loss
'''

selected_layers = ['block1_conv1', 'block2_conv2',"block3_conv1","block3_conv3" ,'block4_conv1',
                   'block4_conv3','block5_conv1','block5_conv2','block5_conv3','block5_conv4']

selected_layer_weights = [1.0, 1.0 , 2.0 , 2.0 ,4.0,
                         4.0 , 8.0 ,8.0 ,16.0, 32.0]

vgg = VGG19(weights='imagenet', include_top=False, input_shape=(256,256,3))
vgg.trainable = False
outputs = [vgg.get_layer(l).output for l in selected_layers]
model = Model(vgg.input, outputs)

@tf.function
def perceptual_loss(input_image , reconstruct_image):
    h1_list = model(input_image)
    h2_list = model(reconstruct_image)

    rc_loss = 0.0

    img = K.batch_flatten(input_image)
    r_img = K.batch_flatten(reconstruct_image)
    r_error =  K.sum(K.abs(img - r_img), axis=-1) /(256 * 256)

    for h1, h2, weight in zip(h1_list, h2_list, selected_layer_weights):

        h1 = K.batch_flatten(h1)
        h2 = K.batch_flatten(h2)
        rc_loss = rc_loss + weight * K.sum(K.abs(h1 - h2), axis=-1)

    rc_loss = (rc_loss / (sum(selected_layer_weights)))
    error = (rc_loss + r_error)/2

    return error 

In [ ]:
rmsprop = RMSprop(learning_rate=0.0001)
AE.compile(loss= perceptual_loss, optimizer= rmsprop)

In [ ]:
es = EarlyStopping(monitor="val_loss",
                   patience=15)

rs = ReduceLROnPlateau(monitor="val_loss",
                  factor=0.1,
                  patience=5,
                  verbose=1,
                  mode="auto")

check = ModelCheckpoint('check.h5')

In [ ]:
root = '../input/siim-isic-melanoma-classification/jpeg/train/'

images = []

for path in tqdm(paths[:2500]):
    path = root + path + '.jpg'
    img = load_img(path,target_size=(256,256))
    img = img_to_array(img)
    img = img.astype(np.float32)
    img = (img)/255.0
    img = np.asarray(img)
    img = img.reshape(1,256,256,3)
    img = data_augmentation(img)
    images.append(img[0])
images = np.asarray(images)

In [ ]:
images.shape

In [ ]:
AE.fit(x=images,y=images,
    epochs=60,
    verbose = 1,
    batch_size = 8,
    validation_split = 0.3,
    callbacks = [es,rs,check])

In [ ]:
del images

In [ ]:
root = '../input/siim-isic-melanoma-classification/jpeg/train/'

images = []

for path in tqdm(paths[2500:5000]):
    path = root + path + '.jpg'
    img = load_img(path,target_size=(256,256))
    img = img_to_array(img)
    img = img.astype(np.float32)
    img = (img)/255.0
    img = np.asarray(img)
    img = img.reshape(1,256,256,3)
    img = data_augmentation(img)
    images.append(img[0])
images = np.asarray(images)

print(images.shape)

AE.fit(x=images,y=images,
    epochs=60,
    verbose = 1,
    batch_size = 8,
    validation_split = 0.3,
    callbacks = [es,rs,check])

AE.save_weights('AE_weights.h5')

del images

In [ ]:
def error(img,r_img):
    img = K.batch_flatten(img)
    r_img = K.batch_flatten(r_img)
    r_error =  K.sum(K.abs(img - r_img), axis=-1)
    return r_error

In [ ]:
paths = train[train.target == 1].image_name.values
root = '../input/siim-isic-melanoma-classification/jpeg/train/'

errors = []
targets = []

for path in tqdm(paths):
    label = train[train.image_name == path].target.values[0]
    targets.append(label)
    path = root + path + '.jpg'
    img = load_img(path,target_size=(256,256))
    img = img_to_array(img)
    img = img.astype(np.float32)
    img = (img)/255.0
    img = np.asarray(img)
    img = img.reshape(1,256,256,3)
    reconstruct = AE(img)
    r_error = error(img[0],reconstruct[0])
    errors.append(r_error) 

In [ ]:
errors[:5]

In [ ]:
targets[:5]

In [ ]:
paths = train[train.target == 0].image_name.values
paths = paths[:584]
root = '../input/siim-isic-melanoma-classification/jpeg/train/'

for path in tqdm(paths):
    label = train[train.image_name == path].target.values[0]
    targets.append(label)
    path = root + path + '.jpg'
    img = load_img(path,target_size=(256,256))
    img = img_to_array(img)
    img = img.astype(np.float32)
    img = (img)/255.0
    img = np.asarray(img)
    img = img.reshape(1,256,256,3)
    reconstruct = AE(img)
    r_error = error(img[0],reconstruct[0])
    errors.append(r_error) 

In [ ]:
errors = np.asarray(errors)
targets = np.asarray(targets)

In [ ]:
errors.shape

In [ ]:
targets.shape

In [ ]:
#target_SVM = targets

In [ ]:
#targets = to_categorical(targets)

In [ ]:
# input_layer = Input(shape=(256), name ="input")

# layer1 = Dense(256 ,activation='relu' , name="layer1-1")(input_layer)
# layer1 = Dense(128 ,activation='relu' , name="layer2-1")(layer1)
# layer1 = Dense(64 ,activation='relu' , name="layer4-1")(layer1)
# layer1 = Dense(32 ,activation='relu' , name="layer5-1")(layer1)
# output = Dense(2 ,activation='softmax' , name="output1")(layer1)

# model1 = Model(inputs=input_layer, outputs=output , name = "Classifier1")
# ######################
# layer2 = Dense(256 ,activation='relu' , name="layer1-2")(input_layer)
# layer2 = Dense(64 ,activation='relu' , name="layer2-2")(layer2)
# layer2 = Dense(64 ,activation='relu' , name="layer3-2")(layer2)
# layer2 = Dense(16 ,activation='relu' , name="layer4-2")(layer2)
# layer2 = Dense(8 ,activation='relu' , name="layer5-2")(layer2)
# output = Dense(2 ,activation='softmax' , name="output2")(layer2)

# model2 = Model(inputs=input_layer, outputs=output , name = "Classifier2")
# ######################
# layer3 = Dense(256 ,activation='relu' , name="layer1-3")(input_layer)
# layer3 = Dense(256 ,activation='relu' , name="layer2-3")(layer3)
# layer3 = Dense(64 ,activation='relu' , name="layer3-3")(layer3)
# layer3 = Dense(16 ,activation='relu' , name="layer4-3")(layer3)
# output = Dense(2 ,activation='softmax' , name="output3")(layer3)

# model3 = Model(inputs=input_layer, outputs=output , name = "Classifier3")
# ######################

# print(model1.summary())
# print(model2.summary())
# print(model3.summary())

In [ ]:
# es = EarlyStopping(monitor="val_loss",
#                    patience=30)

# rs = ReduceLROnPlateau(monitor="val_loss",
#                   factor=0.1,
#                   patience=10,
#                   verbose=1,
#                   mode="auto")

In [ ]:
# rmsprop = RMSprop(learning_rate=0.0001)
# sgd = SGD(learning_rate=0.0001,momentum=0.99)
# model1.compile(loss= 'binary_crossentropy', metrics= [tf.keras.metrics.Accuracy(),tf.keras.metrics.AUC()], optimizer= rmsprop)
# model2.compile(loss= 'binary_crossentropy', metrics= [tf.keras.metrics.Accuracy(),tf.keras.metrics.AUC()], optimizer= sgd)
# model3.compile(loss= 'binary_crossentropy', metrics= [tf.keras.metrics.Accuracy(),tf.keras.metrics.AUC()], optimizer= rmsprop)

In [ ]:
# model1.fit(x=errors,y=targets,
#     epochs=150,
#     verbose = 1,
#     batch_size = 32,
#     validation_split = 0.3,
#     callbacks = [es,rs])

# model1.save_weights('Classifier1_weights.h5')

In [ ]:
# model2.fit(x=errors,y=targets,
#     epochs=150,
#     verbose = 1,
#     batch_size = 64,
#     validation_split = 0.3,
#     callbacks = [es,rs])

# model2.save_weights('Classifier2_weights.h5')

In [ ]:
# model3.fit(x=errors,y=targets,
#     epochs=150,
#     verbose = 1,
#     batch_size = 16,
#     validation_split = 0.3,
#     callbacks = [es,rs])

# model3.save_weights('Classifier3_weights.h5')

In [ ]:
# from sklearn.svm import SVC
# clf = SVC()
# clf.fit(errors, target_SVM)

In [ ]:
parameters = {
    'n_estimators': [1,2,4,8,16,32,64,128,256]}
classifier = GridSearchCV(AdaBoostClassifier(), parameters, scoring='accuracy', n_jobs= 4, cv=5)
classifier.fit(errors, targets)

In [ ]:
AdaBoost = classifier.best_estimator_
print (classifier.best_score_, classifier.best_params_)

In [ ]:
del targets
del errors
#del target_SVM

In [ ]:
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
test.head()

In [ ]:
paths = test.image_name.values
root = '../input/siim-isic-melanoma-classification/jpeg/test/'

# preds1 = []
# preds2 = []
# preds3 = []
preds4 = []

for path in tqdm(paths):
    path = root + path + '.jpg'
    img = load_img(path,target_size=(256,256))
    img = img_to_array(img)
    img = img.astype(np.float32)
    img = (img)/255.0
    img = np.asarray(img)
    img = img.reshape(1,256,256,3)
    reconstruct = AE(img)
    r_error = error(img[0],reconstruct[0]).numpy()
    r_error = r_error.reshape(1,256)
    
#     pred1 = model1(r_error)
#     pred1 = np.argmax(pred1, axis=1)
#     preds1.append(pred1)
    
#     pred2 = model2(r_error)
#     pred2 = np.argmax(pred2, axis=1)
#     preds2.append(pred2)
    
#     pred3 = model3(r_error)
#     pred3 = np.argmax(pred3, axis=1)
#     preds3.append(pred3)
    
    pred4 = AdaBoost.predict(r_error)
    preds4.append(pred4)
    
#preds1 = np.asarray(preds1)
# preds2 = np.asarray(preds2)
# preds3 = np.asarray(preds3)
preds4 = np.asarray(preds4)

In [ ]:
# preds1[:,0]

In [ ]:
# preds2[:,0]

In [ ]:
# preds3[:,0]

In [ ]:
preds4[:,0]

In [ ]:
# preds1.shape

In [ ]:
# preds2.shape

In [ ]:
# preds3.shape

In [ ]:
preds4.shape

In [ ]:
#sub = pd.DataFrame({'image_name':paths , 'target1':preds1[:,0],'target2':preds2[:,0],'target3':preds3[:,0],'target4':preds4[:,0]})
sub = pd.DataFrame({'image_name':paths,'target':preds4[:,0]})
sub.head()

In [ ]:
#sub['target'] = (round((sub['target1'] + sub['target2'] + sub['target3'] + 3*sub['target4'])/6,0))
#sub['target'] = sub['target'].astype('int32')

In [ ]:
#sub = sub[['image_name','target']]
#sub.head()

In [ ]:
sub.to_csv('submission.csv',index = False)